# phase 2


## importing librarie


In [48]:
from hazm import *
from __future__ import unicode_literals
# from parsivar import *
import json
import string
from hazm import stopwords_list
from copy import copy
from matplotlib import pyplot as plt

import math

## Reading Json file

In [49]:
file=open("IR_data_news_12k.json")
data=json.load(file)

## create empty arrays to store contents


In [50]:
data_content=[]
data_url=[]
data_title=[]

## store content in arrays 

In [51]:
for i in range(len(data)):
    index=str(i)
    data_content.append(data[""+index]["content"])
    data_url.append(data[""+index]["url"])
    data_title.append(data[""+index]["title"])

In [52]:
#comment after complete
# data_content=data_content[:10]

## define new array for tokenize

In [53]:
data_tokens=[]

# preproccesing start

## normalize contents
tokenize and stemmer and lematize

In [54]:
normalizer = Normalizer()
stemmer = Stemmer()
stop_words=set(stopwords_list())
stop_words.add('\u200cو')
stop_words.add('\u200cو')
stop_words.add('و\u200c')
lemmatizer = Lemmatizer()

## define punctuations

In [55]:
punctuations=string.punctuation 
temp_list=set()
for i in punctuations:
    temp_list.add(i)
temp_list.add('،')
temp_list.add('.')
temp_list.add(':')
punctuations=temp_list.copy()

### loop over te content and preprocess

In [56]:
for j in range (len(data_content)):
    data_content[j]=normalizer.normalize(data_content[j])
    new_contents=word_tokenize(data_content[j])
    temp_contents=copy(new_contents)
    for k in range(len(temp_contents)):
        if(temp_contents[k] in stop_words):
            new_contents.remove(temp_contents[k])    
        if(temp_contents[k] in punctuations):
            new_contents.remove(temp_contents[k])
#     for k in range(len(new_contents)):
#         new_contents[k]=lemmatizer.lemmatize(new_contents[k])
# #         new_contents[k]=stemmer.stem(new_contents[k])

    data_tokens.append(new_contents)            
            

## define function for find Tf

In [57]:
def find_tf(content,word):
    count=0
    for i in content:
        if(word==i):
            count+=1
      
    return math.log(count)+1


### indexing words 
making words indexing in contents

#### empty set for tokens index

In [58]:
final_token=dict()

## find Tf for each word and document

In [59]:
for i in range(len(data_tokens)):
    for j in range(len(data_tokens[i])):
        if(data_tokens[i][j] in final_token):
            temp_string=final_token[data_tokens[i][j]]
            last_content=temp_string.split("|")
            no=int(last_content[len(last_content)-1].split(":")[0].split("c")[1])
            if(no==i):
                count=last_content[len(last_content)-1].split(":")
                temp_count=int(count[len(count)-1].split("_")[1])
                temp_count+=1
                new_string=""
                for s in range(len(last_content)-1):
                    new_string+=last_content[s]+"|"
#                 print(last_content[len(last_content)-1])
                last_item=last_content[len(last_content)-1].split("_")[0]+"_"+str(temp_count)+"_"
                new_string+=last_item
                final_token[data_tokens[i][j]]=new_string
            else: 
                final_token[data_tokens[i][j]]+="|"+"c"+str(i)+":"+str(find_tf(data_tokens[i],data_tokens[i][j]))+"_1_"
        else:
            final_token[data_tokens[i][j]]="|"+"c"+str(i)+":"+str(find_tf(data_tokens[i],data_tokens[i][j]))+"_1_"
            


##  calculating IDF

##### function for calulating IDF

In [60]:
def idf(string):
    temp_str=len(string.split("|"))-1
    return math.log(len(data_tokens)/temp_str)
        

In [61]:
for i in final_token:
    final_token[i]+="@"+str(idf(final_token[i]))+"@"

### make champion list

In [62]:
temp_token=dict()
for i in final_token:
    string=final_token[i].split("|")[1:]
    idf=final_token[i].split("@")[1]
    new_arr=[]
    if(len(string)>=5):
        for j in range(5):
            maximum=-1
            maxstr=""
            for k in string:
                tf=float(k.split(":")[1].split("_")[0])
                if(maximum<=tf):
                    maximum=tf
                    maxstr=k
            new_arr.append(maxstr)
            string.remove(maxstr)
        ans=""
        for j in new_arr:
            ans+="|"+j
        ans+="@"+idf+"@"    
        temp_token[i]=ans
        
    else:
        temp_token[i]=final_token[i]
            
    

In [63]:
final_token=temp_token

## make vector for each doc

In [64]:
doc_vector=[]


In [65]:
for i in range(len(data_tokens)):
    vector=[]
    sum_tf_idf=0
    for j in range(len(data_tokens[i])):
        term=data_tokens[i][j]
        term_data=final_token[term]
        
        tf_idf=-1
#         print(term_data.split("@")[len(term_data.split("@"))-2]
        idf=float(term_data.split("@")[len(term_data.split("@"))-2])
        term_data=term_data.split("|")[1:]
        for k in term_data:
            doc_no=int(k.split(":")[0].split("c")[1])
            if(i==doc_no):
#                 print(k.split(":")[1].split("_"))
                tf=float(k.split(":")[1].split("_")[0])
                tf_idf=tf*idf
        sum_tf_idf+=(tf_idf*tf_idf)
#         print(term)
#         print(tf_idf)
        vector.append(term+":"+str(tf_idf))  
    sum_tf_idf=math.sqrt(sum_tf_idf)   
    for j in range(len(vector)):
        word=vector[j].split(":")[0]
#         print(vector[j].split(":"))
        if(len(vector[j].split(":"))>2):
                tf_idf=float(vector[j].split(":")[2])
        else:        
            tf_idf=float(vector[j].split(":")[1])
        tf_idf/=sum_tf_idf
        vector[j]=word+":"+str(tf_idf)
        
    doc_vector.append(vector)         

### nomalize

# User input Query
get input from user to proccess query

In [66]:
query=input("please enter your input : ")

please enter your input : صنعت نفت آبادان


##### process input

In [67]:
query=query.split(" ")

In [68]:
query_vector=dict()
tf_sum=0

In [69]:
final_token["آبادان"]

'|c4084:3.833213344056216_17_|c53:3.6390573296152584_14_|c6655:3.5649493574615367_13_|c5930:3.4849066497880004_12_|c4140:3.4849066497880004_12_@3.400541966269203@'

In [70]:
for i in query:
    tf=find_tf(query,i)
    idf=0
    if(i in final_token):
        print(i)
        temp=final_token[i]  
        idf=float(temp.split("@")[1])
        print(idf)
    tf_idf=tf*idf    
    tf_sum+=(tf_idf*tf_idf)
    query_vector[i]=tf_idf
math.sqrt(tf_sum)
if(tf_sum!=0):
    for k in query_vector:
        query_vector[k]/=tf_sum
    

صنعت
2.809484652498961
نفت
2.4071991973081763
آبادان
3.400541966269203


In [71]:
 query_vector

{'صنعت': 0.11126011974264441,
 'نفت': 0.09532896743133366,
 'آبادان': 0.1346669418608397}

In [72]:
docs_ans=[]
docs_ans_no=[]
for j in range(len(data_tokens)):
        ans=0
        doc_no=-1
        for i in query_vector:
            if i in data_tokens[j]:
                counter=1
                for k in range(len(doc_vector[j])):
                    word=doc_vector[j][k].split(":")
                    if(i==word[0]):
                        if(counter==1):
#                             print(word)
#                             print(query_vector[i])
#                             print(word[1])
                            ans+=query_vector[i]*float(word[1])
                            doc_no=j
                            counter-=1
        if(ans!=0):
            docs_ans.append(ans)
            docs_ans_no.append(doc_no)

In [73]:
# docs_ans

In [74]:
# docs_ans_no

# show answers 

In [75]:
final_answer=[]
if(len(docs_ans)==0):
    print("no answer found for your query")  
else:
    if(len(docs_ans)>=5):
        for i in range(5):
            maximum=-10
            max_id=0
            for k in range(len(docs_ans)):
                if(docs_ans[k]>maximum):
                    maximum=docs_ans[k]
                    max_id=docs_ans_no[k]       
            final_answer.append(max_id)
            docs_ans.remove(maximum)
            docs_ans_no.remove(max_id)
    else:
         for i in range(len(docs_ans)):
            maximum=0
            max_id=0
            for k in range(len(docs_ans)):
                if(docs_ans[k]>maximum):
                    maximum=docs_ans[k]
                    max_id=docs_ans_no[k]
            final_answer.append(max_id)
            docs_ans.remove(maximum)
            docs_ans_no.remove(max_id)
    count=0    
    for k in final_answer:
        docid=int(k)
        print("answer "+str(count+1)+": ")
        print("title: "+data_title[docid])
        print("url: "+data_url[docid])
        print(docid)
        print(data_content[docid])
        count+=1

answer 1: 
title: منصوریان: بازیکنان ما ناراحت هستند/ فردا ترکیب متفاوتی از صنعت نفت مقابل آلومینیوم خواهیم دید
url: https://www.farsnews.ir/news/14001007000810/منصوریان-بازیکنان-ما-ناراحت-هستند-فردا-ترکیب-متفاوتی-از-صنعت-نفت
5930

به گزارش خبرگزاری فارس از آبادان، تیم فوتبال صنعت نفت آبادان یکی از بحران‌زده تیم‌های چند سال اخیر لیگ برتر فوتبال کشور است که با وجود انبوهی از مشکلات مالی و زیرساختی، هر ساله به سختی در کشور تیم‌داری می‌کند که این مشکلات، در فصل جاری نیز به اوج خود رسیده است. در حالی که شاگردان منصوریان فردا در هفته دوازدهم در آبادان از حریف اراکی خود پذیرایی خواهند کرد، این باشگاه پس از گذشت ۱۲ هفته هنوز نتوانسته به اعضای تیم پرداختی داشته باشد. نفتی‌های آبادان عصر چهارشنبه در حالی به مصاف شاگردان خطیبی خواهند رفت که در هفته گذشته خبرهایی مبنی بر اعتصاب و تهدید به فسخ قرارداد از سوی برخی از بازیکنان این تیم به گوش رسیده است. در مقابل آلومینیوم اراک که فصل گذشته را با مربیگری منصوریان به پایان رسانده است، پایاپای مدعیان به آبادان آمده است در کورس باقی بماند. بر همین اساس، 

In [ ]:
# final_answer
